In [2]:
import re
import torch
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
import platform
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from sklearn.model_selection import KFold

# Dropout
Dropout을 적용하고 싶은 레이어 전에 Dropout을 넣어줘야 함<br>
train시에는 꺼지지 않은 값이 2배가 됨

In [3]:
x = torch.Tensor([[1,2,3,4], [4,5,6,7]])
nn.Dropout(0.5).forward(x)

tensor([[ 2.,  0.,  6.,  8.],
        [ 8., 10.,  0.,  0.]])

# nn.CrossEntropyLoss()
logsoftmax + nllloss이기 때문에 모델 안에 softmax레이어를 넣을 필요 없음<br>
target은 long이어야 하고 input은 데이터 개수 x output_dim(=카테고리개수)여야 함<br>
output은 데이터개수 차원이고 안의 내용은 카테고리 숫자여야함 <br>

In [4]:
>>> loss = nn.CrossEntropyLoss()
>>> input = torch.randn(3, 5, requires_grad=True)
>>> target = torch.empty(3, dtype=torch.long).random_(5)
>>> output = loss(input, target)
>>> output.backward()

In [5]:
target.shape, input.shape

(torch.Size([3]), torch.Size([3, 5]))

# inplace operation

In [6]:
x.add_(1)
x

tensor([[2., 3., 4., 5.],
        [5., 6., 7., 8.]])

# repeat

In [7]:
>>> x = torch.tensor([1, 2, 3])
>>> x.repeat(4, 2)

tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3]])

In [42]:
torch.arange(0, 10).unsqueeze(0).repeat(3, 1)

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

# bmm
Performs a batch matrix-matrix product of matrices stored in input and mat2.

input and mat2 must be 3-D tensors each containing the same number of matrices.

In [34]:
input = torch.randn(10, 3, 4)
mat2 = torch.randn(10, 4, 5)
res = torch.bmm(input, mat2)
res.size()
torch.Size([10, 3, 5])

torch.Size([10, 3, 5])

# matmul
https://pytorch.org/docs/stable/generated/torch.matmul.html#torch.matmul

broadcasting된다는 것이 bmm과의 차이

In [39]:
input = torch.randn(10, 3, 4)
mat2 = torch.randn(10, 4, 5)
res = input.matmul(mat2)
res.size()

torch.Size([10, 3, 5])

In [40]:
>>> # batched matrix x broadcasted matrix
>>> tensor1 = torch.randn(10, 3, 4)
>>> tensor2 = torch.randn(4, 5)
>>> torch.matmul(tensor1, tensor2).size()

torch.Size([10, 3, 5])

In [41]:
torch.bmm(tensor1, tensor2)

RuntimeError: Expected 3-dimensional tensor, but got 2-dimensional tensor for argument #2 'batch2' (while checking arguments for bmm)

# tril

In [47]:
torch.tril(torch.Tensor(3, 4))

tensor([[1.9052e+05, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [       nan, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.6778e+20, 3.0866e+29, 7.5205e+28, 0.0000e+00]])

# masked_fill

In [60]:
x = torch.arange(10)
mask = x < 5 

In [61]:
mask

tensor([ True,  True,  True,  True,  True, False, False, False, False, False])

In [62]:
x.masked_fill(mask, 3)

tensor([3, 3, 3, 3, 3, 5, 6, 7, 8, 9])

# sentence_bleu

In [9]:
from sacrebleu import sentence_bleu

In [17]:
sentence_bleu(['i love you'], ['i kill you'])

BLEU = 34.67 66.7/25.0/25.0/0.0 (BP = 1.000 ratio = 1.000 hyp_len = 3 ref_len = 3)

# layer norm

In [63]:
>>> input = torch.randn(3, 4)
>>> # With Learnable Parameters
>>> m = nn.LayerNorm(input.size()[1:])

In [13]:
input.shape, m(input).shape

(torch.Size([3, 4]), torch.Size([3, 4]))

In [64]:
m(input).mean(dim = 1)

tensor([-1.4901e-08,  2.9802e-08,  3.7253e-09], grad_fn=<MeanBackward1>)

In [65]:
m(input).std(dim = 1)

tensor([1.1547, 1.1547, 1.1547], grad_fn=<StdBackward1>)

# view vs. transpose

In [66]:
x = torch.arange(100)
x.view(2, 5, 10)

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]],

        [[50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
         [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
         [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
         [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]])

In [67]:
x = torch.arange(100)
x.view(5, 2, 10)

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]],

        [[40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]],

        [[60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74, 75, 76, 77, 78, 79]],

        [[80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
         [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]])

In [68]:
x = torch.arange(10)
xx = x.view(5, 2)

In [69]:
xx

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [70]:
xx.view(2, 5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [71]:
xx.transpose(1, 0)

tensor([[0, 2, 4, 6, 8],
        [1, 3, 5, 7, 9]])